In [8]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from glob import glob
sys.path.append('../../src/')

import classifier.Multiband as multiband


the filtering of the times must be done in the spine, not in each band. Spit all the times then slect the best one. 

In [9]:
# To see if the system regognises the GPU
device = 0
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[device], 'GPU')
tf.config.experimental.set_memory_growth(device=devices[device], enable=True)


Find the different folds and train a model using the stored data.

In [10]:
survey = 'Gaia'
path = os.path.join('../02_CreateRecords/Folds/Fold_*/',)
folds = glob(path)
folds.sort()
folds

['../02_CreateRecords/Folds/Fold_1/',
 '../02_CreateRecords/Folds/Fold_2/',
 '../02_CreateRecords/Folds/Fold_3/']

Create folder results

In [11]:
if not os.path.exists('./Results'):
    os.mkdir('./Results')


Define the arguments for all the models.

In [12]:
train_args = {
            'hidden_size_bands':[128, 128, 128],
            'hidden_size_central':[128, 128],
            'fc_layers_bands':[128,128,128],
            'fc_layers_central':[128,128,128], # Neurons of each layer
            'regression_size':[128, 128],#each element is a layer with that size.
            'buffer_size':10000,
            'epochs':5,
            'num_threads':7,
            'batch_size':1024,
            'dropout':0.40,
            'lr':[[1e-3]*2, 1e-3], # [[band1, band2], central]
            'val_steps':50,
            'max_to_keep':0, # Not Used 
            'steps_wait':0, 
            'use_class_weights':False,# Not Used
            'mode' : 'classifier+regression'
            }
loss_weights = {'Class':300.0, 'T_eff':20.0,'Radius':1e0}

callbacks_args = {'patience': 20,
                  'mode':'max',
                  'restore_best_weights':True,
                  'min_delta': 0.001
                 }
train_args_specific={
                    'phys_params': ['T_eff', 'Radius'],
                    'use_output_bands' : True,  # Working
                    'use_output_central' : False, # Not used
                    'use_common_layers' : False, # NOT Working
                    'bidirectional_central' : False,# Working
                    'bidirectional_band' : False,# Not Working
                    'layer_norm_params' : None, # Used to normalyze common layers
                    'use_gated_common' : False, # Working
                    'l1':0.0,
                    'l2':0.0,   
                    'N_skip' : 8, # Cannot be greater than the number of timesteps
                    'use_raw_input_central': True,
                    'train_steps_central' : 2,
                    'print_report' : True,
                    'loss_weights_central' : loss_weights,
                    'callbacks_args':callbacks_args
                    }



In [13]:
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold+'/'
    
    # Set the save path for this fold. Create folder if needed
    # path_results_fold = fold.replace('../../02_Create_Records/'+survey+'/', './').replace('/Folds/', '/Results/')
    path_results_fold = fold.replace('../02_CreateRecords', '.').replace('/Folds/', '/Results/')

    if not os.path.exists(path_results_fold):
        os.mkdir(path_results_fold)    

    train_args_specific['save_dir'] = path_results_fold
    train_args_specific['metadata_pre_path'] = base_dir+'metadata_preprocess.json'  
    train_args_specific['path_scalers'] =  base_dir+'scalers.pkl'
    # Define the train args
    train_args = {**train_args, **train_args_specific}
    
    train_files = base_dir+'train/*.tfrecord'
    val_files = base_dir+'val/*.tfrecord'
    test_files = base_dir+'test/*.tfrecord'
    
    new = multiband.Network()    
    new.train(train_args, train_files, val_files, test_files)
    new.train_loop()


./Results/Fold_1/Models/20230706-2042


2023-07-06 20:42:26.825558: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2023-07-06 20:42:26.825579: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2023-07-06 20:42:27.151062: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2023-07-06 20:42:27.152028: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2023-07-06 20:42:27.168438: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2023-07-06 20:42:27.168457: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2023-07-06 20:42:27.258896: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2023-07-06 20:42:27.259795: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed


Start training
13/13 [==============================] - 2s 137ms/step - loss: 5207.5293 - Class_loss: 17.3530 - Pred_Radius_loss: 0.9183 - Pred_T_eff_loss: 0.0357 - Class_CentralAcc: 0.6970 - Class_CentralTop2: 0.8817 - Class_FinalAcc: 0.7216 - Class_FinalTop2: 0.8960 - Class_Final_FScore: 0.6347 - Pred_Radius_Masked_RMSE: 0.7236 - Pred_Radius_Masked_R2: 0.6718 - Pred_T_eff_Masked_RMSE: 0.1528 - Pred_T_eff_Masked_R2: 0.8545
Early Stopping
              precision    recall  f1-score   support

         CEP       0.62      0.75      0.68      1367
  DSCT_SXPHE       0.59      0.60      0.60       880
     MIRA_SR       0.90      0.90      0.90      2200
        RRAB       0.74      0.71      0.73      2200
         RRC       0.69      0.70      0.70      2200
       T2CEP       0.37      0.15      0.22       436

    accuracy                           0.72      9283
   macro avg       0.65      0.64      0.64      9283
weighted avg       0.72      0.72      0.72      9283

{'R2': {'T_eff

2023-07-06 21:00:20.180161: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2023-07-06 21:00:20.180181: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2023-07-06 21:00:20.502625: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2023-07-06 21:00:20.503595: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2023-07-06 21:00:20.520860: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2023-07-06 21:00:20.520879: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2023-07-06 21:00:20.613385: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2023-07-06 21:00:20.614282: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed


Start training
13/13 [==============================] - 2s 131ms/step - loss: 5153.3486 - Class_loss: 17.1724 - Pred_Radius_loss: 0.9057 - Pred_T_eff_loss: 0.0361 - Class_CentralAcc: 0.6968 - Class_CentralTop2: 0.8853 - Class_FinalAcc: 0.7181 - Class_FinalTop2: 0.8995 - Class_Final_FScore: 0.6194 - Pred_Radius_Masked_RMSE: 0.7175 - Pred_Radius_Masked_R2: 0.6800 - Pred_T_eff_Masked_RMSE: 0.1540 - Pred_T_eff_Masked_R2: 0.8479
Early Stopping
              precision    recall  f1-score   support

         CEP       0.64      0.71      0.67      1367
  DSCT_SXPHE       0.55      0.61      0.58       880
     MIRA_SR       0.90      0.91      0.90      2200
        RRAB       0.74      0.72      0.73      2200
         RRC       0.67      0.72      0.70      2200
       T2CEP       0.30      0.08      0.13       436

    accuracy                           0.72      9283
   macro avg       0.64      0.62      0.62      9283
weighted avg       0.71      0.72      0.71      9283

{'R2': {'T_eff

2023-07-06 21:17:57.264491: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2023-07-06 21:17:57.264513: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2023-07-06 21:17:57.400704: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2023-07-06 21:17:57.401576: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2023-07-06 21:17:57.419260: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2023-07-06 21:17:57.419289: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2023-07-06 21:17:57.547354: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2023-07-06 21:17:57.548231: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed


Start training
13/13 [==============================] - 2s 160ms/step - loss: 5190.3726 - Class_loss: 17.2956 - Pred_Radius_loss: 0.9304 - Pred_T_eff_loss: 0.0374 - Class_CentralAcc: 0.6937 - Class_CentralTop2: 0.8816 - Class_FinalAcc: 0.7175 - Class_FinalTop2: 0.8939 - Class_Final_FScore: 0.6336 - Pred_Radius_Masked_RMSE: 0.7302 - Pred_Radius_Masked_R2: 0.6637 - Pred_T_eff_Masked_RMSE: 0.1573 - Pred_T_eff_Masked_R2: 0.8454
Early Stopping
              precision    recall  f1-score   support

         CEP       0.65      0.69      0.67      1367
  DSCT_SXPHE       0.57      0.60      0.58       880
     MIRA_SR       0.90      0.91      0.90      2200
        RRAB       0.72      0.71      0.72      2200
         RRC       0.67      0.70      0.68      2200
       T2CEP       0.34      0.19      0.24       436

    accuracy                           0.72      9283
   macro avg       0.64      0.63      0.63      9283
weighted avg       0.71      0.72      0.71      9283

{'R2': {'T_eff